In [9]:
import pandas as pd

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
data = pd.read_csv("SIH Data.csv")
data['Day'] = data['Day'].ffill()

In [12]:
data['Datetime'] = pd.to_datetime(data['Day'] + ' ' + data['Time'], format='%b-%d %H:%M')
data.set_index('Datetime', inplace=True)
data.drop(columns=['Day', 'Time'], inplace=True)
data = data.dropna(how='any',axis=0)


In [13]:
train_size = 0.8  # 80% of data for training, 20% for testing
split_index = int(len(data) * train_size)

# Splitting the data chronologically
train, test = data[:split_index], data[split_index:]

In [14]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

ts = train['Load (MW)']
exog_train = train[['Temp', 'Rain(mm)', 'Gust(km/hr)', 'Rain%', 'IsHoliday']]
exog_test = test[['Temp', 'Rain(mm)', 'Gust(km/hr)', 'Rain%', 'IsHoliday']]

model = SARIMAX(ts, exog=exog_train, order=(5, 1, 0))
model_fit = model.fit()

forecast = model_fit.get_forecast(steps=len(test), exog=exog_test)
forecast_series = pd.Series(forecast.predicted_mean.values, index=test.index)

In [15]:
for z,i in zip(test.index[-4:], forecast.predicted_mean.values[-4:]):
    print(f"{z} \t {i:.2f}")

1900-08-31 09:00:00 	 5323.01
1900-08-31 12:00:00 	 5736.45
1900-08-31 15:00:00 	 5999.72
1900-08-31 21:00:00 	 5626.36


In [16]:
for z,i in zip(test.index[-4:], forecast.predicted_mean.values[-4:]):
    print(f"{i:.2f}")

5323.01
5736.45
5999.72
5626.36
